# Exploring Interrelations between Classes
We can assume that given classes toxic and severe_toxic, having one class implies having the other. What about other target variables - does one variable affect another and can we use it to improve the final score? This notebook is an attempt to glimpse into the decision-making process of the annotators and also in the nature of the data itself. We will calculate one standard and one not-so-standard statistic and visualize them using plotly heatmaps.

In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

First, we load the dataset and separate text data from the target variables.

In [2]:
train = pd.read_csv('../input/train.csv')
train.index = train['id']
x_train = train['comment_text']
y_train = train.iloc[:, 2:]

Then, let's calculate Spearman correlation between the target variables.

In [8]:
correlation_matrix = y_train.corr()

..and plot it using a heatmap.

In [36]:
heatmap = go.Heatmap(
    z=np.flip(correlation_matrix.values, axis=1),  # try it without flipping - looks unusual
    x=y_train.columns[::-1],
    y=y_train.columns,
    showscale=False,
    colorscale="viridis"
)

layout = go.Layout(
    title="Correlation between target variables",
    showlegend=False,
    width=700, height=700,
    autosize=False,
    margin=go.Margin(l=100, r=100, b=100, t=100, pad=4)
)

fig = go.Figure(data=[heatmap], layout=layout)
iplot(fig, filename='hmap')

We see that, for instance, obscene and insult really correlate with each other, wich is kind of logical - most insults are obscene. But this doesn't give us the full picture. What is really important is, according to annotators, do we always assign "toxic" for "sever_toxic"? And what about "obscene"?

Let's calculate a following matrix: given classes *i* and *j*, what proportion of objects belonging to class *i* also belong to class *j*?

In [54]:
dim = y_train.shape[1]
cooccurence_matrix = np.zeros((dim, dim))

for i in range(dim):
    for j in range(dim):
        res = sum(y_train.iloc[:, i] & y_train.iloc[:, j]) / sum(y_train.iloc[:, i])
        cooccurence_matrix[i, j] = res

And another heatmap:

In [55]:
heatmap = go.Heatmap(
    z=np.flip(cooccurence_matrix, axis=1),
    x=y_train.columns[::-1],
    y=y_train.columns,
    showscale=False,
    colorscale="viridis"
)

layout = go.Layout(
    title="Coocurence of target variables",
    showlegend=False,
    width=700, height=700,
    autosize=False,
    margin=go.Margin(l=100, r=100, b=100, t=100),
    yaxis=dict(
        title='If...'
    ),
    xaxis=dict(
        title='then...'
    )
)

fig = go.Figure(data=[heatmap], layout=layout)
iplot(fig, filename='hmap')

What this means is *if we have severe_toxic, we have toxic 100% of the time* (hover over the plot to see the figures). This means it is a good sanity check to see if your sever_toxic comments also have toxic class. Also threats are toxic in 95% and obscenities are toxic in 94% of the cases.

If a comment is an insult, it is obscene in 78% of the cases, which is, as already said, quite logical. More interestingly, 86% of sever_toxic comments also contains insults, as well as 82% of identity_hate instances. But if a comment is obscene, it is a threat only 3% of the time, but if it is a threat, it more probably than not contains an insult (65%).